<a href="https://colab.research.google.com/github/reypapin/Generador-de-Dominios/blob/main/LLama2_7B_Domain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 3.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 8.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 10.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 6.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-stmvi3ob/overlay/local/bin
    changing mode of /tmp/pip-build-env-stmvi3ob/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-stmvi3ob/overlay/loca

In [4]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin" # the model is in bin format
# test zephyr in format GGUF instead of GGML

In [5]:
from huggingface_hub import hf_hub_download

In [6]:
from llama_cpp import Llama

In [7]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-7b-chat.ggmlv3.q4_0.bin:   0%|          | 0.00/3.79G [00:00<?, ?B/s]

In [8]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [9]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [11]:
import pandas as pd

# Assuming the file is named "domain_generated.csv""
df = pd.read_csv("domain_genered.csv")

# Now you can work with the DataFrame df as you wish
print(df)

                  DGA
0         iisy.com.au
1     oolwsnrt.online
2             ma.club
3      bsoevell.co.jp
4       oittar.com.au
...               ...
9995        pselhh.cc
9996      sida.com.au
9997       ct-lowd.se
9998            ae.co
9999      iaassuac.it

[10000 rows x 1 columns]


In [13]:
import pandas as pd

# Assuming the file is named "domain_generated.csv"
df = pd.read_csv("domain_genered.csv")

# Get the first 100 domain names from the DataFrame df
domain_names = df['DGA'].iloc[0:10].tolist()

# List to store the responses
responses = []

# Iterate over each domain name
for domain_name in domain_names:
    # Construct the prompt for the current domain name
    prompt = f"I want to classify the following domain name '{domain_name}' as 'DGA' (algorithmically generated) or 'normal' and I want you to respond with a 1 if it is dga and a 0 if it is normal, I want your response as follows: The domain is 1, or The domain is 0."

    prompt_template = f'''SYSTEM: Domain name classification.

USER: {prompt}

ASSISTANT:
'''

    # Generate response for the current domain name
    response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.0, top_p=0.95,
                        repeat_penalty=1.2, top_k=150, echo=True)

    # Add the response to the list of responses
    responses.append(response["choices"][0]["text"])

# Print the responses
for i, response in enumerate(responses):
    print(f"Response for batch {i+1}:\n{response}\n")


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Response for batch 1:
SYSTEM: Domain name classification.

USER: I want to classify the following domain name 'iisy.com.au' as 'DGA' (algorithmically generated) or 'normal' and I want you to respond with a 1 if it is dga and a 0 if it is normal, I want your response as follows: The domain is 1, or The domain is 0.

ASSISTANT:
Thank you for providing the domain name 'iisy.com.au'. Based on my analysis, I would classify this domain name as 'normal' (0). Here's why:

The domain name 'iisy.com.au' contains a mix of letters and numbers, which is common for many normal domain names. It does not contain any obvious algorithmic patterns or structures that are typically found in DGA (Domain Generation Algorithm) domains. Additionally, the top-level domain '.au' is also a common TLD (top-level domain) used by many countries around the world, and it does not have any distinctive features that would indicate it as an DGA domain.

Therefore, based on my analysis, I would classify 'iisy.com.au' as a

In [14]:

# Variables to store the total count of '0's, '1's, and undecided responses
total_0 = 0
total_1 = 0
total_no_decision = 0

# Iterate over each response in the responses variable
for response in responses:
    # Find the position of 'ASSISTANT' in the response
    assistant_index = response.find('ASSISTANT')

    # Extract the response after 'ASSISTANT'
    extracted_response = response[assistant_index + len('ASSISTANT'):].strip()

    # Flag to check if '0' or '1' was found in the response
    found_decision = False

    # Search for '0' or '1' in the response
    for char in extracted_response:
        if char == '0':
            total_0 += 1
            found_decision = True
            break
        elif char == '1':
            total_1 += 1
            found_decision = True
            break

    # Increment the counter if no decision was found
    if not found_decision:
        total_no_decision += 1

# Print the total counters
print("Total number of '0's in all responses:", total_0)
print("Total number of '1's in all responses:", total_1)
print("Total number of undecided responses:", total_no_decision)



Total number of '0's in all responses: 9
Total number of '1's in all responses: 1
Total number of undecided responses: 0
